In [1]:
import pandas as pd
import numpy as np
import sys
import os

In [2]:
df = pd.read_csv("/content/major-crime-indicators.csv")

In [3]:
df

,_id,EVENT_UNIQUE_ID,REPORT_DATE,OCC_DATE,REPORT_YEAR,REPORT_MONTH,REPORT_DAY,REPORT_DOY,REPORT_DOW,REPORT_HOUR,...,UCR_CODE,UCR_EXT,OFFENCE,MCI_CATEGORY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,LONG_WGS84,LAT_WGS84
0,1,GO-20141263217,2014-01-01,2013-12-31,2014,January,1,1,Wednesday,16,...,2135,210,Theft Of Motor Vehicle,Auto Theft,043,Victoria Village (43),043,Victoria Village (43),-79.306754,43.734654
1,2,GO-20141260715,2014-01-01,2014-01-01,2014,January,1,1,Wednesday,3,...,1430,100,Assault,Assault,092,Corso Italia-Davenport (92),092,Corso Italia-Davenport (92),-79.455770,43.677775
2,3,GO-20141260730,2014-01-01,2014-01-01,2014,January,1,1,Wednesday,3,...,1430,100,Assault,Assault,105,Lawrence Park North (105),105,Lawrence Park North (105),-79.406223,43.727681
3,4,GO-20141260597,2014-01-01,2014-01-01,2014,January,1,1,Wednesday,2,...,1430,100,Assault,Assault,080,Palmerston-Little Italy (80),080,Palmerston-Little Italy (80),-79.415594,43.654946
4,5,GO-20141259762,2014-01-01,2013-12-31,2014,January,1,1,Wednesday,2,...,1430,100,Assault,Assault,164,Wellington Place (164),077,Waterfront Communities-The Island (77),-79.390786,43.649125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420195,420196,GO-20242816627,2024-12-31,2024-12-24,2024,December,31,366,Tuesday,10,...,2135,210,Theft Of Motor Vehicle,Auto Theft,033,Clanton Park (33),033,Clanton Park (33),-79.449706,43.734062
420196,420197,GO-20242815828,2024-12-31,2024-12-30,2024,December,31,366,Tuesday,7,...,2135,210,Theft Of Motor Vehicle,Auto Theft,025,Glenfield-Jane Heights (25),025,Glenfield-Jane Heights (25),-79.513210,43.739385
420197,420198,GO-20242814680,2024-12-31,2024-12-31,2024,December,31,366,Tuesday,2,...,1610,200,Robbery - Mugging,Robbery,168,Downtown Yonge East (168),075,Church-Yonge Corridor (75),-79.378304,43.656258
420198,420199,GO-20242815237,2024-12-31,2024-12-31,2024,December,31,366,Tuesday,4,...,2120,200,B&E,Break and Enter,032,Englemount-Lawrence (32),032,Englemount-Lawrence (32),-79.433324,43.719406


In [4]:
columns_to_keep = [
    'OCC_YEAR',
    'NEIGHBOURHOOD_158',
    'LAT_WGS84',
    'LONG_WGS84',
    'MCI_CATEGORY'
]

In [5]:
start_year_threshold = 2014

In [6]:
initial_rows = len(df)
df.dropna(subset=['NEIGHBOURHOOD_158', 'LAT_WGS84', 'LONG_WGS84', 'OCC_YEAR'], inplace=True)
rows_after_na = len(df)
if initial_rows > rows_after_na:
    print(f"Removed {initial_rows - rows_after_na} rows with missing essential data.")


Removed 6426 rows with missing essential data.


In [7]:
len(df)

413774

In [8]:
df['OCC_YEAR'] = pd.to_numeric(df['OCC_YEAR'], errors='coerce')
df.dropna(subset=['OCC_YEAR'], inplace=True) # Drop rows where conversion failed
df['OCC_YEAR'] = df['OCC_YEAR'].astype(int)

In [ ]:
len(df)

412318

In [12]:
df_filtered = df[df['OCC_YEAR'] >= start_year_threshold].copy()
print(f"Filtered dataset has {df_filtered.shape[0]} rows.")

Filtered dataset has 412318 rows.


In [14]:
df_filtered.rename(columns={
    'NEIGHBOURHOOD_158': 'Neighbourhood',
    'LAT_WGS84': 'Latitude',
    'LONG_WGS84': 'Longitude',
    'MCI_CATEGORY': 'MCI_Category',
    'OCC_YEAR': 'Year'
}, inplace=True)

In [13]:
columns_to_keep = ['Neighbourhood','Latitude','Longitude','MCI_Category','Year']

In [15]:
df_filtered = df_filtered[columns_to_keep]

In [16]:
df_filtered

,Neighbourhood,Latitude,Longitude,MCI_Category,Year
1,Corso Italia-Davenport (92),43.677775,-79.455770,Assault,2014
2,Lawrence Park North (105),43.727681,-79.406223,Assault,2014
3,Palmerston-Little Italy (80),43.654946,-79.415594,Assault,2014
5,West Rouge (143),43.780413,-79.132915,Assault,2014
6,West Rouge (143),43.780413,-79.132915,Assault,2014
...,...,...,...,...,...
420195,Clanton Park (33),43.734062,-79.449706,Auto Theft,2024
420196,Glenfield-Jane Heights (25),43.739385,-79.513210,Auto Theft,2024
420197,Downtown Yonge East (168),43.656258,-79.378304,Robbery,2024
420198,Englemount-Lawrence (32),43.719406,-79.433324,Break and Enter,2024


In [17]:
output_csv_path = '/content/major-crime-indicators.csv'

In [18]:
df_filtered.to_csv(output_csv_path, index=False)

In [19]:
if os.path.exists(output_csv_path):
    file_size_bytes = os.path.getsize(output_csv_path)
    file_size_mb = file_size_bytes / (1024 * 1024)
    print(f"Output file size: {file_size_mb:.2f} MB")

Output file size: 29.58 MB


In [20]:
df = df_filtered

In [21]:
df['MCI_Category'].unique()

array(['Assault', 'Break and Enter', 'Theft Over', 'Robbery',
       'Auto Theft'], dtype=object)

In [22]:
yearly_counts = df.groupby("Year", as_index=False).size()

In [23]:
yearly_counts

,Year,size
0,2014,32009
1,2015,32631
2,2016,33157
3,2017,34979
4,2018,36982
5,2019,39252
6,2020,34731
7,2021,34338
8,2022,40976
9,2023,48523


In [24]:
df["Neighbourhood"] = df["Neighbourhood"].str.replace(r"\s*\(\d+\)", "", regex=True)

In [25]:
df

,Neighbourhood,Latitude,Longitude,MCI_Category,Year
1,Corso Italia-Davenport,43.677775,-79.455770,Assault,2014
2,Lawrence Park North,43.727681,-79.406223,Assault,2014
3,Palmerston-Little Italy,43.654946,-79.415594,Assault,2014
5,West Rouge,43.780413,-79.132915,Assault,2014
6,West Rouge,43.780413,-79.132915,Assault,2014
...,...,...,...,...,...
420195,Clanton Park,43.734062,-79.449706,Auto Theft,2024
420196,Glenfield-Jane Heights,43.739385,-79.513210,Auto Theft,2024
420197,Downtown Yonge East,43.656258,-79.378304,Robbery,2024
420198,Englemount-Lawrence,43.719406,-79.433324,Break and Enter,2024


In [27]:
mci_map = {cat: i+1 for i, cat in enumerate(sorted(df["MCI_Category"].unique()))}
df["MCI_Category_ID"] = df["MCI_Category"].map(mci_map)

In [28]:
mci_map

{'Assault': 1,
 'Auto Theft': 2,
 'Break and Enter': 3,
 'Robbery': 4,
 'Theft Over': 5}

In [30]:
mci_map_df = pd.DataFrame(list(mci_map.items()), columns=["MCI_Category", "ID"])
mci_map_df.to_csv("/content/mci_map.csv", index=False)

In [31]:
mci_map_df

,MCI_Category,ID
0,Assault,1
1,Auto Theft,2
2,Break and Enter,3
3,Robbery,4
4,Theft Over,5


In [32]:
neigh_map = {name: i+1 for i, name in enumerate(sorted(df["Neighbourhood"].unique()))}

In [33]:
neigh_map

{'Agincourt North': 1,
 'Agincourt South-Malvern West': 2,
 'Alderwood': 3,
 'Annex': 4,
 'Avondale': 5,
 'Banbury-Don Mills': 6,
 'Bathurst Manor': 7,
 'Bay-Cloverhill': 8,
 'Bayview Village': 9,
 'Bayview Woods-Steeles': 10,
 'Bedford Park-Nortown': 11,
 'Beechborough-Greenbrook': 12,
 'Bendale South': 13,
 'Bendale-Glen Andrew': 14,
 'Birchcliffe-Cliffside': 15,
 'Black Creek': 16,
 'Blake-Jones': 17,
 'Briar Hill-Belgravia': 18,
 'Bridle Path-Sunnybrook-York Mills': 19,
 'Broadview North': 20,
 'Brookhaven-Amesbury': 21,
 'Cabbagetown-South St.James Town': 22,
 'Caledonia-Fairbank': 23,
 'Casa Loma': 24,
 'Centennial Scarborough': 25,
 'Church-Wellesley': 26,
 'Clairlea-Birchmount': 27,
 'Clanton Park': 28,
 'Cliffcrest': 29,
 'Corso Italia-Davenport': 30,
 'Danforth': 31,
 'Danforth East York': 32,
 'Don Valley Village': 33,
 'Dorset Park': 34,
 'Dovercourt Village': 35,
 'Downsview': 36,
 'Downtown Yonge East': 37,
 'Dufferin Grove': 38,
 'East End-Danforth': 39,
 "East L'Amoreau

In [34]:
df["Neighbourhood_ID"] = df["Neighbourhood"].map(neigh_map)

In [35]:
neigh_map_df = pd.DataFrame(list(neigh_map.items()), columns=["Neighbourhood", "ID"])
neigh_map_df.to_csv("/content/neighbourhood_map.csv", index=False)

In [36]:
df

,Neighbourhood,Latitude,Longitude,MCI_Category,Year,MCI_Category_ID,Neighbourhood_ID
1,Corso Italia-Davenport,43.677775,-79.455770,Assault,2014,1,30
2,Lawrence Park North,43.727681,-79.406223,Assault,2014,1,81
3,Palmerston-Little Italy,43.654946,-79.415594,Assault,2014,1,110
5,West Rouge,43.780413,-79.132915,Assault,2014,1,143
6,West Rouge,43.780413,-79.132915,Assault,2014,1,143
...,...,...,...,...,...,...,...
420195,Clanton Park,43.734062,-79.449706,Auto Theft,2024,2,28
420196,Glenfield-Jane Heights,43.739385,-79.513210,Auto Theft,2024,2,54
420197,Downtown Yonge East,43.656258,-79.378304,Robbery,2024,4,37
420198,Englemount-Lawrence,43.719406,-79.433324,Break and Enter,2024,3,45


In [53]:
mapped_df = df[[
    "Year",
    "Neighbourhood_ID",
    "MCI_Category_ID",
    "Latitude",
    "Longitude",
]]

In [54]:
mapped_df

,Year,Neighbourhood_ID,MCI_Category_ID,Latitude,Longitude
1,2014,30,1,43.677775,-79.455770
2,2014,81,1,43.727681,-79.406223
3,2014,110,1,43.654946,-79.415594
5,2014,143,1,43.780413,-79.132915
6,2014,143,1,43.780413,-79.132915
...,...,...,...,...,...
420195,2024,28,2,43.734062,-79.449706
420196,2024,54,2,43.739385,-79.513210
420197,2024,37,4,43.656258,-79.378304
420198,2024,45,3,43.719406,-79.433324


In [39]:
mapped_df.to_csv(output_csv_path, index=False)

In [40]:
if os.path.exists(output_csv_path):
    file_size_bytes = os.path.getsize(output_csv_path)
    file_size_mb = file_size_bytes / (1024 * 1024)
    print(f"Output file size: {file_size_mb:.2f} MB")

Output file size: 18.59 MB


In [55]:
unique_latlong = mapped_df[["Latitude", "Longitude"]].drop_duplicates()

In [56]:
print("Unique (Latitude, Longitude) pairs:", len(unique_latlong))

Unique (Latitude, Longitude) pairs: 19223


In [59]:
unique_latlong = df[["Latitude", "Longitude"]].drop_duplicates().reset_index(drop=True)

In [60]:
len(unique_latlong)

19223

In [61]:
unique_latlong["LatLong_ID"] = unique_latlong.index + 1

In [63]:
mapped_df = mapped_df.merge(unique_latlong, on=["Latitude", "Longitude"], how="left")

In [67]:
mapped_df["LatLong_ID"].isna().sum()

np.int64(0)

In [68]:
mapped_df.drop(columns=["Latitude", "Longitude"], inplace=True)

In [69]:
unique_latlong.to_csv("/content/latlong_id_map.csv", index=False)

In [70]:
mapped_df.to_csv(output_csv_path, index=False)

In [ ]:
#now lets check all file sizes

In [78]:
def check_file_size(file_path:str):
  if os.path.exists(file_path):
    file_size_bytes = os.path.getsize(file_path)
    file_size_mb = file_size_bytes / (1024 * 1024)
    return file_size_mb

In [73]:
csv_files = [file for file in os.listdir("/content/") if file.endswith('.csv')]

for file in csv_files:
    print(file)

neighbourhood_map.csv
latlong_id_map.csv
major-crime-indicators.csv
mci_map.csv


In [80]:
total_size = 0

In [81]:
for file in csv_files:
  total_size += check_file_size(file)
print(f"total Output file size: {total_size:.2f} MB")

total Output file size: 6.78 MB


In [ ]:
#  well we finally got it under 10!

In [84]:
mapped_df

,Year,Neighbourhood_ID,MCI_Category_ID,LatLong_ID
0,2014,30,1,1
1,2014,81,1,2
2,2014,110,1,3
3,2014,143,1,4
4,2014,143,1,4
...,...,...,...,...
412313,2024,28,2,12192
412314,2024,54,2,128
412315,2024,37,4,269
412316,2024,45,3,6809


In [85]:
unique_latlong

,Latitude,Longitude,LatLong_ID
0,43.677775,-79.455770,1
1,43.727681,-79.406223,2
2,43.654946,-79.415594,3
3,43.780413,-79.132915,4
4,43.686293,-79.486285,5
...,...,...,...
19218,43.633507,-79.507854,19219
19219,43.781159,-79.129854,19220
19220,43.700742,-79.400431,19221
19221,43.760778,-79.418256,19222
